<a href="https://colab.research.google.com/github/somwrks/AI-Solar-Panel/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Self Rotatory Solar Panel

This Project utilizes various vision models to train on planet sun labelled datasets and perform validation checks to determine which model performs the best and converts the best model to tinyML Model to make it usable by microcontroller

## Installing Dependencies

In [ ]:
%pip install --upgrade pip


In [ ]:
%pip install  tensorflow[and-cuda] tensorflow-datasets roboflow ultralytics torch torchvision matplotlib tqdm


## Importing Libraries

In [18]:
import tensorflow as tf
import tensorflow_datasets as tfds
import torch
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader, ConcatDataset
from ultralytics import YOLO
import matplotlib.pyplot as plt
from tqdm import tqdm
from roboflow import Roboflow

import yaml

import os

from PIL import Image


In [19]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # Suppress TensorFlow logging
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'  # Disable oneDNN custom operations


In [20]:
print("TensorFlow version:", tf.__version__)
print("GPU Available:", tf.test.is_gpu_available())


TensorFlow version: 2.18.0
GPU Available: True


I0000 00:00:1735021126.584486   28208 gpu_device.cc:2022] Created device /device:GPU:0 with 3539 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4050 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.9


In [21]:
# List available GPUs
physical_devices = tf.config.list_physical_devices('GPU')
print("Num GPUs Available: ", len(physical_devices))

if len(physical_devices) > 0:
    # Optimize memory growth
    for gpu in physical_devices:
        tf.config.experimental.set_memory_growth(gpu, True)
    
    # Set visible devices (if you have multiple GPUs and want to use a specific one)
    tf.config.set_visible_devices(physical_devices[0], 'GPU')
    
    # Enable mixed precision training
    from tensorflow.keras import mixed_precision
    policy = mixed_precision.Policy('mixed_float16')
    mixed_precision.set_global_policy(policy)

    print("GPU optimization settings applied successfully")
else:
    print("No GPU available. Running on CPU.")

Num GPUs Available:  1
GPU optimization settings applied successfully


## Loading Datasets

In [22]:
rf = Roboflow(api_key="")
project = rf.workspace("1009727588-qq-com").project("sun-nxvfz")
roboflow_dataset = project.version(2).download("yolov5")

# Prepare dataset paths
dataset_path = roboflow_dataset.location
train_path = os.path.join(dataset_path, 'train')
valid_path = os.path.join(dataset_path, 'valid')
test_path = os.path.join(dataset_path, 'test')

# Create YAML file for YOLOv5
yaml_content = {
    'train': train_path,
    'val': valid_path,
    'test': test_path,
    'nc': len(os.listdir(os.path.join(dataset_path, 'train', 'labels'))),
    'names': [f'class_{i}' for i in range(len(os.listdir(os.path.join(dataset_path, 'train', 'labels'))))]
}

yaml_path = os.path.join(dataset_path, 'dataset.yaml')
with open(yaml_path, 'w') as yaml_file:
    yaml.dump(yaml_content, yaml_file, default_flow_style=False)


loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to sun-2 in yolov5pytorch:: 100%|██████████| 342/342 [00:01<00:00, 291.94it/s]


## Setting Up Models


### 2. YoloV5 (LibRT)

In [23]:
# Load a pretrained YOLOv5 model
yolo_model = YOLO('yolov5s.pt')

# Train YOLOv5 on Roboflow dataset
yolo_results = yolo_model.train(data=yaml_path, epochs=50, imgsz=640)


PRO TIP 💡 Replace 'model=yolov5s.pt' with new 'model=yolov5su.pt'.
YOLOv5 'u' models are trained with https://github.com/ultralytics/ultralytics and feature improved performance vs standard YOLOv5 models trained with https://github.com/ultralytics/yolov5.

Ultralytics 8.3.53 🚀 Python-3.12.3 torch-2.5.1+cu124 CUDA:0 (NVIDIA GeForce RTX 4050 Laptop GPU, 6140MiB)
engine/trainer: task=detect, mode=train, model=yolov5s.pt, data=/mnt/c/Users/Som/Desktop/AI-Solar-Panel/sun-2/dataset.yaml, epochs=50, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_d

train: Scanning /mnt/c/Users/Som/Desktop/AI-Solar-Panel/sun-2/train/labels... 115 images, 0 backgrounds, 0 corrupt: 100%|██████████| 115/115 [00:00<00:00, 509.76it/s]


train: New cache created: /mnt/c/Users/Som/Desktop/AI-Solar-Panel/sun-2/train/labels.cache


val: Scanning /mnt/c/Users/Som/Desktop/AI-Solar-Panel/sun-2/valid/labels... 32 images, 0 backgrounds, 0 corrupt: 100%|██████████| 32/32 [00:00<00:00, 261.19it/s]


val: New cache created: /mnt/c/Users/Som/Desktop/AI-Solar-Panel/sun-2/valid/labels.cache
Plotting labels to /mnt/c/Users/Som/Desktop/AI-Solar-Panel/runs/detect/train2/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=8.4e-05, momentum=0.9) with parameter groups 69 weight(decay=0.0), 76 weight(decay=0.0005), 75 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to /mnt/c/Users/Som/Desktop/AI-Solar-Panel/runs/detect/train2
Starting training for 50 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...


      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       3/50      3.94G       2.15      8.528      1.796          5        640: 100%|██████████| 8/8 [01:25<00:00, 10.67s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:27<00:00, 27.31s/it]

                   all         32         32      0.123       0.25     0.0934     0.0452



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       4/50      3.97G      2.041      7.374      1.735         19        640:  62%|██████▎   | 5/8 [00:21<00:14,  4.69s/it]

In [ ]:
# Evaluate YOLOv5
yolo_metrics = yolo_model.val()

print("YOLOv5 mAP:", yolo_metrics.results_dict['metrics/mAP50-95(B)'])


## Testing Models

In [ ]:
# tbd

## Saving Model

In [ ]:
model.save('sun_tracker_model.h5')
